In [4]:
def hexa2bin(hex):
    # Convert a hexadecimal bit stream to a binary bit stream
    
    hex_dict = {'0': '0000', '1': '0001', '2': '0010', '3': '0011', '4': '0100', '5': '0101', '6': '0110', '7': '0111', '8': '1000', '9': '1001', 'a': '1010', 'b': '1011', 'c': '1100', 'd': '1101', 'e': '1110', 'f': '1111'}
    
    bin = ''
    for c in hex:
         bin += hex_dict[c]
    
    return bin

def bin2dec(bin):
    # Convert a binary bit stream to its decimal representation
    # Returns -1 if a empty string is passed
    
    if(bin == ''):
        return -1
    return int(bin, 2)

In [6]:
def get_domain_name(dns, start):
    # Gets the domain name given a dns packet
    # Returns the domain name, the start of the answer section and the number of octets in the
    # question section. This is required to parse the answer section in the get_ip function
    
    domain_name = ''
    
    num_octets = bin2dec(dns[start : start + 8])
    if(num_octets == -1):
        return -1, -1, -1
    
    start += 8
    total_octets = 0

    while(num_octets != 0):
        if(num_octets > 63):
            return -1, -1, -1
        total_octets += (num_octets + 1)
        for i in range(num_octets):
            octet = bin2dec(dns[start : start + 8])
            if not((octet == 45) or (octet > 47 and octet < 58) or (octet > 64 and octet < 91) or (octet > 96 and octet < 123)):
                return -1, -1, -1
            domain_name += chr(octet)
            start += 8

        num_octets = bin2dec(dns[start : start + 8])
        if(num_octets == -1):
            return -1, -1, -1
        
        start += 8
        if(num_octets != 0): domain_name += '.'

    total_octets += 1
    start += 4 * 8 
        
    return domain_name, start, total_octets

In [8]:
def get_ip(dns, start, q_octets, domain_name):
    # Returns the ip address from a dns packet, given the packet is a response packet
    # Returns -1 in case of error
    if(start >= len(dns)):
        return -1
        
    if(dns[start] == '1' and dns[start + 1] == '1'):
        # This case corresponds to when the compression algorithm is used to 
        # add a pointer to the domain name in the question section
        len_start = start + 10 * 8
    elif(dns[start] == '0' and dns[start + 1] == '0'):
        # This case corresponds to the case when no compression is involved
        len_start = start + (q_octets + 8) * 8
        answer_domain_name = get_domain_name(dns, start)
        if(answer_domain_name != domain_name):
            return -1
    else:
        # Invalid case. Returns -1
        return -1
        
    num_ip_octets = bin2dec(dns[len_start : len_start + 16])
    if(num_ip_octets == -1):
        return -1

    ip = ''
    ip_start = len_start + 16

    for i in range(num_ip_octets):
        if(i != 0):
            ip += "."
        ip_ = bin2dec(dns[ip_start : ip_start + 8])
        if(ip_ == -1):
            return -1
            
        ip += str(ip_)
        ip_start += 8

    return ip

In [16]:
def parse_dns(dns):
    # Parse a given dns packet. Prints "Invalid DNS Packet" if the packet is invalid
    
    z = bin2dec(dns[25:28])
    rcode = bin2dec(dns[28:32])
    if(rcode or z):
        print("Invalid DNS Packet")
        return
        
    qr = dns[16] # 17th bit of the DNS packet represents if the packet is a question or a response packet
    if(qr == '0'):
        # If the bit is 0, it is a question packet
        domain_name, _, _ = get_domain_name(dns, 96)
        if(domain_name == -1):
            # Error hnadling
            print("Invalid DNS Packet")
            return
            
        print(f"This is a query packet.\nQueried Domain Name: {domain_name}")
    else:
        # If the bit is 1, it is an answer packet
        domain_name, answer_start, total_octets = get_domain_name(dns, 96)
        if(domain_name == -1):
            # Error handling
            print("Invalid DNS Packet")
            return
            
        ip = get_ip(dns, answer_start, total_octets, domain_name)
        if ip == -1:
            # Error handling
            printf("Invalid DNS Packet")
            return
            
        print(f"This is a response packet.\nQueried Domain Name: {domain_name}\nResponse IP Address: {ip}")

In [26]:
dns_packet = input("Enter DNS Packet: ")

Enter DNS Packet:  a0208180000100010000000105666f6e74730a676f6f676c656170697303636f6d00 00010001c00c00010001000000ae00048efab64a0000290200000000000000


In [27]:
dns_packet = ''.join(dns_packet.split())

In [28]:
parse_dns(hexa2bin(dns_packet))

This is a response packet.
Queried Domain Name: fonts.googleapis.com
Response IP Address: 142.250.182.74
